In [1]:
#將原始電子麻單 轉成excel檔
#讀取有txt(這個list是儲存json檔的位置)   輸出csv
import os
import pandas as pd
import numpy as py
import matplotlib.pyplot as plt
from pathlib import Path               #用來清除檔名
import json
from pandas import json_normalize      #用來讀檔案
from tqdm.notebook import tqdm, trange 


#都先設成一個dataframe
df_patient = pd.DataFrame()
df_PreInduction = pd.DataFrame()
df_labData = pd.DataFrame()
df_ABG = pd.DataFrame()
df_induction = pd.DataFrame()
df_infusionLines = pd.DataFrame()
df_facilities = pd.DataFrame()
df_bolusDrugOrders = pd.DataFrame()
df_gasOrders = pd.DataFrame()
df_fluidOrders = pd.DataFrame()
df_bloodOrders = pd.DataFrame()
df_workshifts = pd.DataFrame()
df_observeData = pd.DataFrame()
df_observedDataEntries = pd.DataFrame()
df_memoDrugOrders = pd.DataFrame()
df_infusionDrugOrders = pd.DataFrame()
df_UrineInputOrders = pd.DataFrame()
df_machineData =  pd.DataFrame()
df_bloodGasData = pd.DataFrame()
df_periOper = pd.DataFrame()
df_skinHurt = pd.DataFrame()

#自訂義同時間資廖整合 helperfunction
def helper(x): 
    x = x.dropna()
    if x.empty: 
        return 
    else: 
        return x[0]
    
#設定display展現方式

pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

b = open('GE_TIVA_BIS_list.txt','r',encoding="UTF-8")
out = b.read()
ge_list = json.loads(out) 

filename_list=[]    #寫出檔案時,檔名命名需要用到

for k in ge_list:
    file_name = Path(k).stem      
    filename_list.append(file_name)
    
    
file_counter=0  #檔名命名用

progress = tqdm(total = len(ge_list))

for i in ge_list:
    f = open(i,'r',encoding="utf-8")
    data = f.read()
    js = json.loads(data)
    f.close
    info = json.loads(data)
    

    
    #建立每個麻單有的dataframe
    for key in info.keys():
        #display(key)
        vars()['df_'+ key] = json_normalize(info[key])
    
    #1-PATIENT
    df_Patient=df_patient  #可做的處理,設定index,把手術名稱提出來.
    if df_Patient.empty == False: 
        df_p_s = pd.DataFrame()
        
        if len(df_Patient["schedules"][0])!= 0: 
            df_p_s=df_p_s.append(df_Patient["schedules"][0][0] ,ignore_index=True,sort=False)  #將schedules取出   
            df_Patient=pd.concat([df_Patient,df_p_s],axis = 1,sort= False)
        else:
            pass
        for colum in df_Patient.columns:
            df_Patient = df_Patient.rename(columns={colum:"patient_"+ colum})
    else:
        pass
    
    #2-PREINDUCTION
    df_preinduction=df_PreInduction 
    if df_preinduction.empty == False:
        for colum in df_preinduction.columns:
            df_preinduction = df_preinduction.rename(columns={colum:"preinduction_"+ colum})
    else:
        pass
    
    #3-LABDATA
    df_Labdata=df_labdata
    if df_Labdata.empty == False:
        for colum in df_Labdata.columns:  
            df_colum=pd.DataFrame()
            df_colum=df_colum.append(df_Labdata[colum][0],sort=False,ignore_index=True) 
            df_Labdata = pd.concat([df_Labdata,df_colum] ,axis=1)
            df_Labdata=df_Labdata.rename(columns={'reportAt':colum+'_reportAT','value':colum +'_value'})
            df_Labdata=df_Labdata.drop(colum,axis=1)
    else:
        pass
    
    #4-ABG
    df_abg=df_ABG
    if df_abg.empty == False:
        df_abg=df_abg.append(df_abg['items'][0],ignore_index=True)
        df_abg=df_abg.drop(columns=["items"])
        df_abg['examAt'].fillna(method='ffill', inplace=True)
        for colum in df_abg.columns:
            df_abg=df_abg.rename(columns={colum:'abg_'+ colum})
    else:
        pass
    
    #5-INDUCTION
    df_Induction=df_induction
    if df_Induction.empty == False:
        df_drugs=pd.DataFrame()
        for k in range(len(df_Induction['drugs'][0])):
            df_drugs=df_drugs.append(df_Induction["drugs"][0][k],ignore_index=True,sort=False)

        for colum in df_drugs.columns:
            df_drugs=df_drugs.rename(columns={colum:'drugs_'+ colum})
    
        df_Induction=pd.concat([df_Induction,df_drugs],axis=1,sort= False)
        for colum in df_Induction.columns:
            df_Induction=df_Induction.rename(columns={colum:'induction_'+ colum})
    else:
        pass
    
    #6-INFUSIONLINE
    df_infusionlines =df_infusionLines
    if df_infusionlines.empty == False:
        for colum in df_infusionlines.columns:
            df_infusionlines = df_infusionlines.rename(columns={colum:"infusionlines_"+colum})
    else:
        pass
    
    #7-FACILITIES
    df_Facilities = df_facilities
    if df_Facilities.empty == False:
        for colum in df_Facilities.columns:
            df_Facilities = df_Facilities.rename(columns={colum:"facilities_"+ colum})
    else:
        pass
    
    #8-BOLUSDRUGORDER
    df_bolusdrugorders=df_bolusDrugOrders
    if df_bolusdrugorders.empty == False:
        df_bolusdrugorders=df_bolusdrugorders.set_index("orderAt",drop=False)
        df_bolusdrugorders=df_bolusdrugorders.sort_index(axis=0)

        #分組
        df_name=df_bolusdrugorders.groupby(['name'])         #以name分組
        df_bcon = pd.DataFrame()        

        for key in df_name.groups.keys():
            df_tmp=df_name.get_group(key)
            #給每種藥命名
            for colum in df_tmp.columns:
                df_tmp=df_tmp.rename(columns={colum:'bolusdrugorder_'+ colum+'_'+key})
       
            df_bcon=df_bcon.append(df_tmp)
        
        #重新排序時間
        df_bolusdrugorders=df_bcon.sort_index(axis=0)
        #再依照時間groupby
        df_bolusdrugorders = df_bolusdrugorders.groupby("orderAt")[df_bolusdrugorders.columns].aggregate(helper)
    else:
        pass
    
    #9-GASORDER
    df_gasorders=df_gasOrders
    if df_gasorders.empty == False:
        pass
    else:
        pass
    
    #10-FLUIDORDERS
    df_fluidorders=df_fluidOrders
    if df_fluidorders.empty == False:
        df_fluidorders=df_fluidorders.set_index("beginAt",drop=False)
        #分組
        df_name=df_fluidorders.groupby(['lineDescription'])
        df_fcon = pd.DataFrame()        

        for key in df_name.groups.keys():
            df_tmp=df_name.get_group(key)
            #給每種藥命名
            for colum in df_tmp.columns:
                df_tmp=df_tmp.rename(columns={colum:'fluidorder_'+ colum+'_'+key})
            df_fcon=df_fcon.append(df_tmp)
        #重新排序時間
        df_fluidorders=df_fcon.sort_index(axis=0)
        #再依照時間groupby
        df_fluidorders = df_fluidorders.groupby("beginAt")[df_fluidorders.columns].aggregate(helper)
    else:
        pass

    #11-BLOODORDERS
    df_bloodorders=df_bloodOrders
    if df_bloodorders.empty == False:
        df_bloodorders=df_bloodOrders.set_index('beginAt',drop=False)
    else:
        pass
    
    #12-WORKSHIFTS
    df_Workshifts=df_workshifts
    if df_Workshifts.empty == False:
        for colum in df_Workshifts.columns:
            df_Workshifts = df_Workshifts.rename(columns={colum:"workshifts_"+ colum})
    else:
        pass
    
    #13-OBSERVEDATA
    df_observedata=df_observeData
    if df_observedata.empty == False:
        df_observedata=df_observedata.set_index("observeAt",drop=False)
        #分組
        df_name=df_observedata.groupby(['name'])
        df_obcon = pd.DataFrame()        
        for key in df_name.groups.keys():
            df_tmp=df_name.get_group(key)
            #給每種藥命名
            for colum in df_tmp.columns:
                df_tmp=df_tmp.rename(columns={colum:'observedata_'+ colum+'_'+key})
            df_obcon=df_obcon.append(df_tmp)
        #重新排序時間
        df_observedata=df_obcon.sort_index(axis=0)
        #再依照時間groupby
        df_observedata = df_observedata.groupby("observeAt")[df_observedata.columns].aggregate(helper)
    else:
        pass
    
    #14-OBSERVEDATAENTRIES
    df_observeddataentries=df_observedDataEntries
    if df_observeddataentries.empty == False:
        for colum in df_observeddataentries.columns:
            df_observeddataentries = df_observeddataentries.rename(columns={colum:"observeddataentries_"+ colum})
    else:
        pass

    #15-MEMODRUGORDER
    df_memodrugorders=df_memoDrugOrders
    if df_memodrugorders.empty == False:
        df_memodrugorders=df_memoDrugOrders.set_index("orderAt",drop=False)
        #分組
        df_name=df_memodrugorders.groupby(['name']) 
        df_mcon = pd.DataFrame()   
        for key in df_name.groups.keys():
            df_tmp=df_name.get_group(key)
            #給每種藥命名
            for colum in df_tmp.columns:
                df_tmp=df_tmp.rename(columns={colum:'memodrugorder_'+ colum+'_'+key})
            df_mcon=df_mcon.append(df_tmp)
        #重新排序時間
        df_memodrugorders=df_mcon.sort_index(axis=0)
        #再依照時間groupby
        df_memodrugorders = df_memodrugorders.groupby("orderAt")[df_memodrugorders.columns].aggregate(helper)
    else:
        pass

    #16-MACHINEDATA
    df_machinedata=df_machineData
    if df_machinedata.empty == False:
        df_machinedata=df_machinedata.set_index("datetime",drop=True)
        df_machinedata=df_machinedata.sort_index(axis=0)
        #display(df_machinedata)

        #改欄位名
        for colum in df_machinedata.columns:
            df_machinedata=df_machinedata.rename(columns={colum:'machinedata_'+ colum})

        df_machinedata = df_machinedata.groupby("datetime")[df_machinedata.columns].aggregate(helper)
    else:
        pass
    
    #17-BLOODGASDATA
    df_bloodgasdata=df_bloodGasData
    if df_bloodgasdata.empty == False:
        df_bloodgasdata=df_bloodGasData.set_index("datetime",drop=False)
        #分組
        df_name=df_bloodgasdata.groupby(['item'])
        df_bgcon = pd.DataFrame()   
        for key in df_name.groups.keys():
            df_tmp=df_name.get_group(key)
            #給每種藥命名
            for colum in df_tmp.columns:
                df_tmp=df_tmp.rename(columns={colum:'bloodgasdata_'+ colum+'_'+key})
            df_bgcon=df_bgcon.append(df_tmp)
        #重新排序時間
        df_bloodgasdata=df_bgcon.sort_index(axis=0)
        #再依照時間groupby
        df_bloodgasdata = df_bloodgasdata.groupby("datetime")[df_bloodgasdata.columns].aggregate(helper)
    else:
        pass
    
    #18-INFUSIONDRUGORDER
    df_infusiondrugorders=df_infusionDrugOrders
    if df_infusiondrugorders.empty == False:
        df_infusiondrugorders=df_infusiondrugorders.set_index("orderAt",drop=False)
        #分組
        df_name=df_infusiondrugorders.groupby(['name'])
        df_icon = pd.DataFrame()   
        for key in df_name.groups.keys():
            df_tmp=df_name.get_group(key)
        #給每種藥命名
            for colum in df_tmp.columns:
                df_tmp=df_tmp.rename(columns={colum:'infusiondrugorder_'+ colum+'_'+key})
            df_icon=df_icon.append(df_tmp)
        #重新排序時間
        df_infusiondrugorders=df_icon.sort_index(axis=0)
        #再依照時間groupby
        df_infusiondrugorders = df_infusiondrugorders.groupby("orderAt")[df_infusiondrugorders.columns].aggregate(helper)
    else:
        pass   
    
    #19-URINEINPUTORDER
    df_urine=df_UrineInputOrders
    if df_urine.empty == False:
        df_urine=df_urine.set_index("orderAt",drop=False)
        #分組
        df_name=df_urine.groupby(['name'])
        df_ucon = pd.DataFrame()   
        for key in df_name.groups.keys():
            df_tmp=df_name.get_group(key)
            #給每種藥命名
            for colum in df_tmp.columns:
                df_tmp=df_tmp.rename(columns={colum:'urine_'+ colum+'_'+key})
            df_ucon=df_ucon.append(df_tmp)
        #重新排序時間
        df_urine=df_ucon.sort_index(axis=0)
        #再依照時間groupby
        df_urine = df_urine.groupby("orderAt")[df_urine.columns].aggregate(helper)
    else:
        pass

    #20-PERIOPER
    df_perioper=df_periOper
    if df_perioper.empty == False:
        for colum in df_perioper.columns:
            df_perioper=df_perioper.rename(columns={colum:'perioper_'+ colum})
    else:
        pass
    
    #21-SKINHURT
    df_skinhurt=df_skinHurt
    if df_skinhurt.empty == False:
        for colum in df_skinhurt.columns:
            df_skinhurt=df_skinhurt.rename(columns={colum:'skinhurt_'+ colum})
    else:
        pass
    
    #合併1
    df_con1=pd.concat([df_Patient,df_preinduction,df_Labdata,df_abg,
                   df_Induction,df_infusionlines,df_Facilities,df_observeddataentries,
                   df_Workshifts,df_perioper,df_skinhurt],
                   keys=['#Patient','#Preinduction','#Labdata','#ABG',
                         '#Induction','#Infusionlines','#Facilities','#ObservedDataentries',
                         '#Workshifts','#Perioper','#SkinHurt'],
                  axis=1,sort=False)
    
    #合併2
    df_con2=pd.concat([df_machinedata,df_observedata,df_bolusdrugorders,df_gasorders,df_fluidorders,
                   df_bloodorders,df_memodrugorders,df_infusiondrugorders,df_urine,df_bloodgasdata],sort=False)

    df_con2=df_con2.sort_index(axis=0)  #時間先排序
    df_con2=df_con2.groupby(df_con2.index)[df_con2.columns].aggregate(helper)
    
    out_path = "D:\\IL\\gebis_sheet"+"\\"+ filename_list[file_counter] +".xlsx"
    
    with pd.ExcelWriter(out_path) as writer:  
        df_con1.to_excel(writer, sheet_name='Sheet_name_1')
        df_con2.to_excel(writer, sheet_name='Sheet_name_2')
    
    file_counter = file_counter + 1
    progress.update(1) 
            

    
    
    

    


    
    

    

    
    
    


    
    







KeyboardInterrupt: 

In [2]:
#好像這個麻單檔案有問題,有error你們再把這個麻單拿掉
import json
b = open('GE_TIVA_BIS_list.txt','r',encoding="UTF-8")
out = b.read()
ge_list = json.loads(out)
ge_list[6140]

'D:\\IL\\F21358_All\\EAF93C8A2773F50F516FC1864FB3A13F5AF4AFFF6E774FDCB58A4935CE55AF2F2CC5F92C62E346CBBC55B09ACF4D3ADB_201903211340.txt'